# Clean up Rusty's script

#### Rusty's full script

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Feb 25 14:38:51 2015

@author: jamesb
"""

from Tkinter import *
import psycopg2
import os
import numpy as np
import h5py
from scipy import stats
import shutil
import argparse
import csv

In [ ]:
missing_in_LIMS = []
In_LIMS = []

def moveIgorspecimen(specimen,missing_in_LIMS,In_LIMS):
    # Get the 1-sec sweeps for f-I curve et al.
    conn = psycopg2.connect('host=limsdb2 dbname=lims2 user=limsreader password=limsro')
    cur = conn.cursor()
    
    cur.execute("SELECT s.name, s.ephys_roi_result_id, s.id FROM specimens s WHERE s.name LIKE %s", ('%' + specimen,))
    result = cur.fetchone()
    if result is None:
        #print "Could not find specimen result for " + specimen + ". Skipping..."
        missing_in_LIMS.append(specimen)
        return missing_in_LIMS  
    #print "Specimen: " + result[0]
    #print "EphysRoiResult: " + str(result[1])
    global specimen_name
    specimen_name = result[0]
    ephys_roi_result_id = result[1]
    specimen_id = result[2]
    #specimen_hd5 = 
    
    cur.execute("SELECT f.filename, f.storage_directory FROM well_known_files f \
                 WHERE f.attachable_type = 'EphysRoiResult' AND f.attachable_id = %s AND f.filename LIKE '%%pxp'", 
                 (ephys_roi_result_id,))
    result = cur.fetchone()
    #print result
    if result is None:
        #print "Could not find pxp file for " + specimen + ". Skipping..."
        missing_in_LIMS.append(specimen)
        return missing_in_LIMS
    WinP = LinuxtoWindow(result[1])
    pxp_path = WinP + result[0]
    #hdf5_path =
    #print "Pxp file: " + pxp_path
    #pxp_dest = "C:/Rusty_LIMS_check/IgorTemp/" + specimen + ".pxp"
    In_LIMS.append(specimen)
    return In_LIMS
    # shutil.copyfile(pxp_path,pxp_dest)

In [ ]:
def LinuxtoWindow(Linuxpath):
    SplitLinux = Linuxpath.split('/')
    #I should be able to link to \\titan, but I can't figure it out, so I mapped //titan/cns to y:    
    #WindowsP = os.path.join('y:/','mousecelltypes',SplitLinux[4],SplitLinux[5],'')
    WindowsP ="//" + os.path.join('titan','cns','mousecelltypes',SplitLinux[4],SplitLinux[5],'')    
    return (WindowsP)

In [ ]:
def get_filepaths(directory):
    """
    This function will generate the file names in a directory
    tree by walking the tree either top-down or bottom-up. For each
    directory in the tree rooted at directory top (including top itself),
    it yields a 3-tuple (dirpath, dirnames, filenames).
    """
    file_paths = []  # List which will store all of the full filepaths.
 
    # Walk the tree.
    for root, directories, files in os.walk(directory):
        for filename in files:
            # Join the two strings in order to form the full filepath.
            #filepath = os.path.join(root, filename)
           
            file_paths.append(filename)  # Add it to the list.
 
    return file_paths  # Self-explanatory.

In [ ]:
# Run the above function and store its results in a variable. 
full_file_paths = get_filepaths("//allen/programs/celltypes/workgroups/279/Patch-Seq")
#full_file_paths = get_filepaths("Z:/Patch-Seq/")
#full_file_paths = get_filepaths("Z:/MIES experiments/")
#this needs to be (Netowrk:/AIBSDATA/279/MIES experiments) might have to connect similar to LinuxtoWindow function
#Try (\\\\AIBSDATA\\279\\MIES experiments)
#print 'all file paths',full_file_paths

In [ ]:
#Finds all experiments with an associated 4x tif image file; only completed experiments have a 4x tif file so it leaves out incomplete experiments
def find_exp_files(full_file_paths):
    exp_files = []
    for f in full_file_paths:
        if f.endswith('.pxp'): 
            exp_files.append(f)
    return exp_files

all_exp_files = find_exp_files(full_file_paths)
#print 'all exp files', all_exp_files
'''
with open('C:/Rusty_LIMS_check/all_exp_files.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=' ',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in all_exp_files:
        spamwriter.writerow([i])
'''

In [ ]:
#Trims down all file names, human or mouse to just specimen name
def trim_exp_files(all_exp_files):
    exp_list_1 = []
    for f in all_exp_files:
        if f.startswith('H1'):
            exp_list_1.append(f)
        else:
            exp_list_1.append(f[-19:])
    return exp_list_1
trimmed_files = trim_exp_files(all_exp_files)
#print 'trimmed files', trimmed_files

In [ ]:
#Yields list of all human and mouse specimen numbers for every successful experiment for conducting database comparison
def exp_list(trimmed_files):
    exp_list = []
    for f in trimmed_files:
        if f.startswith('H1'):
            if len(f) == 26:
                exp_list.append(f[:22])
            else:
                if len(f) == 23:
                    exp_list.append(f[:19])
        else:
            exp_list.append(f[:15])
    return exp_list
experiment_list = exp_list(trimmed_files)
#print 'experiment list', experiment_list 

for i in experiment_list:
    moveIgorspecimen(row[0],missing_in_LIMS,In_LIMS)

#csv writer: writes specimen names to csv file
#with open('C:/Rusty_LIMS_check/igor.csv', 'wb') as csvfile:
#    spamwriter = csv.writer(csvfile, delimiter=' ',
#                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
#    for i in experiment_list:
#        spamwriter.writerow([i])
#csv reader: reads specimen names from csv file
#with open("C:/Rusty_LIMS_check/igor.csv", 'rb') as csvfile:
#    igorreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
#    for row in igorreader:
#        moveIgorspecimen(row[0],missing_in_LIMS,In_LIMS)
    
#Gives list of all experiments that were not found in LIMS
print "Missing in LIMS", missing_in_LIMS
#print "In LIMS", In_LIMS

In [ ]:
# creates list of folders that have at least one specimen missing in LIMS(will contain repeats where a folder has more than one experiment that did not upload to LIMS)
def folders_missing_in_LIMS(specimens):
    missing_in_LIMS_folders = []
    for specimen in missing_in_LIMS:
        if specimen.startswith('H1'):
            missing_in_LIMS_folders.append(specimen[:10])
        else:
            missing_in_LIMS_folders.append(specimen[:6])
    return missing_in_LIMS_folders

folders_notIn_LIMS = folders_missing_in_LIMS(missing_in_LIMS)
#print "folders not in LIMS", folders_notIn_LIMS

#Returns ordered list of folders that contain experiments that did not upload to LIMS without repeats
folders_notIn_LIMS = list(set(folders_notIn_LIMS))
folders_notIn_LIMS.sort()
print "folders not in LIMS", folders_notIn_LIMS 

In [ ]:
#Returns list of folders already in destination directory
def lst_dest_dir(directory):
    dest_dir_lst = []
    for root, directories, files in os.walk(directory):
        for foldername in directories:
            dest_dir_lst.append(foldername)
    return dest_dir_lst
dest_directory = lst_dest_dir('F:/EphysData/Patch-Seq/')
#print 'destination directory', dest_directory

In [ ]:
def folders_in_LIMS(specimens):
    In_LIMS_folders = []
    for specimen in In_LIMS:
        if specimen.startswith("H1"):
            In_LIMS_folders.append(specimen[:10])
        else:
            In_LIMS_folders.append(specimen[:6])
    return In_LIMS_folders

folders_InLIMS = folders_in_LIMS(In_LIMS)
#print 'folders in LIMS', folders_InLIMS

#Returns ordered list of folders, without repeats, in which all experiments containted within were found in LIMS
folders_InLIMS = list(set(folders_InLIMS))
folders_InLIMS.sort()
#print 'folders in LIMS', folders_InLIMS

In [ ]:
#Defines folders to move - only the folders that are not also in the "folders_notIn_LIMS" list and "dest_directory" list
def folders_to_move(folders_InLIMS):
    In_LIMS_folders_toMove = []
    for folder in folders_InLIMS:
        if folder not in folders_notIn_LIMS and folder not in dest_directory:
            In_LIMS_folders_toMove.append(folder)
    return In_LIMS_folders_toMove
    
folders_that_move = folders_to_move(folders_InLIMS)
print "folders that move", folders_that_move

#Attaching pathway to folders that will move directories
folders_that_move_dirs = ["//allen/programs/celltypes/workgroups/279/Patch-Seq/" + x for x in folders_that_move]
#print "folder that move directories", folders_that_move_dirs

dest_dir = "F:/EphysData/Patch-Seq/"

#for directory in folders_that_move_dirs:
#    try:
#        shutil.move(directory, dest_dir)
#    except (OSError, IOError), e:
#        print >>sys.stderr, 'Error moving %s to %s: %s' % (directory, dest_dir, e)
                       
#print 'Moved_list', os.listdir("F:/EphysData/Patch-Seq/")

In [ ]:
#csv writer to write specimen names of files missing in LIMS to csv file
with open('C:/Rusty_LIMS_check/not_in_LIMS_ps.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=' ',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in missing_in_LIMS:
        spamwriter.writerow([i])